**Draft Model for Mice Steep Stage Analysis**

In [1]:
import torch
import torch.nn.functional as F
from torch import optim, nn
from torchvision import transforms, datasets, models


import torch.utils.data as utils
from inputMassager import *


In [2]:

data_filepath = R"C:\Users\regan\Downloads\CHDCtrl1_CHD801FR_normal\CHD801FR_20221123_normal.txt"
annotated_filepath = R"C:\Users\regan\Downloads\CHDCtrl1_CHD801FR_normal\CHD801FR_20221123_normal_annotated.txt"
period_size = 200

labels, eeg_samples, emg_samples, fft_eeg, fft_emg = get_labeled_data(data_filepath, annotated_filepath, period_size)

In [3]:
ds = torch.utils.data.TensorDataset(eeg_samples, emg_samples, labels)

train_size = int(len(ds) *.80)
val_size = len(ds) - int(len(ds) *.80)

train_dataset, val_dataset = torch.utils.data.random_split(ds, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Creating class for Model**

In [5]:
class Model_3(nn.Module):
    def __init__(self, period_size, input_channels=1):
        super(Model_3, self).__init__()
        self.input_channels = input_channels

        self.eeg_conv1 = nn.Conv1d(in_channels=1, out_channels=period_size, kernel_size=period_size//2)
        self.eeg_conv2 = nn.Conv1d(in_channels=period_size, out_channels=period_size//4, kernel_size=8)
        self.eeg_conv3 = nn.Conv1d(in_channels=period_size // 4, out_channels=256, kernel_size=2)

        self.emg_conv1 = nn.Conv1d(in_channels=1, out_channels=period_size, kernel_size=period_size//2)
        self.emg_conv2 = nn.Conv1d(in_channels=period_size, out_channels=period_size//4, kernel_size=8)
        self.emg_conv3 = nn.Conv1d(in_channels=period_size // 4, out_channels=256, kernel_size=2)



        # out_channels and kernal size are random
        self.fc1 = nn.Linear(2560*2, 20) # 10 is random
        self.fc2 = nn.Linear(10, 5)


    def forward(self, c1, c2):
        #print(x.size())
        c1 = self.eeg_conv1(c1)
        c1 = F.relu(c1)
        # print(x.size())
        c1 = F.max_pool1d(c1, kernel_size=2)
        c1 = self.eeg_conv2(c1)
        c1 = F.max_pool1d(c1, kernel_size=2)
        # print(x.size())
        c1 = self.eeg_conv3(c1)
        c1 = F.max_pool1d(c1, kernel_size=2)
        #print(x.size())

        c2 = self.eeg_conv1(c2)
        c2 = F.relu(c2)
        # print(x.size())
        c2 = F.max_pool1d(c2, kernel_size=2)
        c2 = self.eeg_conv2(c2)
        c2 = F.max_pool1d(c2, kernel_size=2)
        # print(x.size())
        c2 = self.eeg_conv3(c2)
        c2 = F.max_pool1d(c2, kernel_size=2)


        c1c2 = torch.cat((c1, c2), dim=1)

        c1c2 = c1c2.flatten(1)
        #print(x.size())
        c1c2 = self.fc1(c1c2)
        c1c2 = F.max_pool1d(c1c2, kernel_size=2)
        # print(x.size())
        c1c2 = self.fc2(c1c2)
        c1c2 = F.log_softmax(c1c2, dim=1)
#ME      
        #c1c2 = F.relu(c1c2)
        
        return c1c2

In [6]:
def train_model(epochs, model):
    model.train() # set model to training mode
    loss_fun = nn.CrossEntropyLoss() #define a loss function object

    for epoch in range(epochs):

      for batch_idx, (channel1, channel2, target) in enumerate(train_loader):
          #print(c)
          # print(target)
          channel1, channel2, target = channel1.to(device), channel2.to(device), target.to(device)
          
          optimizer.zero_grad()
          output = model(channel1, channel2) # guess we have to pass all channels instead of data?
          loss = loss_fun(output,target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(channel1), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))

In [7]:
model_step_3 = Model_3(period_size)
model_step_3.to(device)
  
optimizer = torch.optim.SGD(model_step_3.parameters(), lr=.01, momentum=0.9)

train_model(3, model_step_3)

Train Epoch: 0 [0/80711 (0%)]	Loss: 1.561570
Train Epoch: 0 [10000/80711 (12%)]	Loss: 0.421642
Train Epoch: 0 [20000/80711 (25%)]	Loss: 0.357577
Train Epoch: 0 [30000/80711 (37%)]	Loss: 0.501190
Train Epoch: 0 [40000/80711 (50%)]	Loss: 0.463170
Train Epoch: 0 [50000/80711 (62%)]	Loss: 0.357382
Train Epoch: 0 [60000/80711 (74%)]	Loss: 0.644982
Train Epoch: 0 [70000/80711 (87%)]	Loss: 0.411984
Train Epoch: 0 [80000/80711 (99%)]	Loss: 0.298962
Train Epoch: 1 [0/80711 (0%)]	Loss: 0.517049
Train Epoch: 1 [10000/80711 (12%)]	Loss: 0.504108
Train Epoch: 1 [20000/80711 (25%)]	Loss: 0.320191
Train Epoch: 1 [30000/80711 (37%)]	Loss: 0.412834
Train Epoch: 1 [40000/80711 (50%)]	Loss: 0.426761
Train Epoch: 1 [50000/80711 (62%)]	Loss: 0.338812
Train Epoch: 1 [60000/80711 (74%)]	Loss: 0.397986
Train Epoch: 1 [70000/80711 (87%)]	Loss: 0.485170
Train Epoch: 1 [80000/80711 (99%)]	Loss: 0.288392
Train Epoch: 2 [0/80711 (0%)]	Loss: 0.302624
Train Epoch: 2 [10000/80711 (12%)]	Loss: 0.342800
Train Epoch: 2 

In [20]:
def evaluate_model(model, dataloader, is_test=False, confidence_level = -0.5):
  # Set model to evaluation mode
  model.eval()

  #dictionary to store the accuracy of predictions by sleep stage
  acurracy_stages = {0:[], 1:[], 2:[], 3:[], 4:[]}
  stage_names = ["not sure(0): ", "rem accuracy(1): ", "non-rem accuracy(2): ", "wake accuracy(3): ", "artifact accuracy(4): "]

  with torch.no_grad():
    correct = 0
    loss = 0

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    for channel1, channel2, target in dataloader:
        channel1, channel2, target = channel1.to(device), channel2.to(device), target.to(device)
        outputs = model(channel1, channel2)

        loss += torch.sum(criterion(outputs, target)).item()

        pred_value, pred = outputs.data.max(1, keepdim=True) # get the index of the max log-probability                                                         
          
        for i in range(len(target)):      
          
          #if the predicted value is lower that the confidence level, set the predicted value to 0 and the target value to 0
           if pred_value[i] < confidence_level:
            pred[i] = 0
            target[i] = 0

          #append boolean indicating whether or not each prediction matched the target to the apporpriate sleep stage list
           acurracy_stages[target[i].item()].append(pred[i].eq(target[i]).item())
      
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()               
                
#calculate accuracy for each stage
    i = 0
    for key in acurracy_stages.keys():

       #acurracy_stages[key] = [acurracy_stages[key].count(True), (acurracy_stages[key].count(True) + acurracy_stages[key].count(False))]
       stage_correct = str(acurracy_stages[key].count(True))
       stage_total = str(acurracy_stages[key].count(True) + acurracy_stages[key].count(False))
       stage_accuracy = str(int(100. * (int(stage_correct) / int(stage_total))))

       print(stage_names[i] + stage_correct + "/" + stage_total + " (" + stage_accuracy + "%)")
       i += 1



#calculate total accuracy and loss
    loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        "Test" if is_test else "Validation",
        loss, correct, len(dataloader.dataset),
        accuracy))

  # Set model back to training mode
  model.train()

In [21]:
#if you make the confidence level too low, you'll get an error about not being able to divede by 0,
#that just means that none of the data is being classified to not sure, keep it below -.75 to avoid this

evaluate_model(model_step_3, train_loader, is_test=True, confidence_level= -.75)

not sure(0): 8/8 (100%)
rem accuracy(1): 0/3032 (0%)
non-rem accuracy(2): 9477/23326 (40%)
wake accuracy(3): 53245/53342 (99%)
artifact accuracy(4): 0/1003 (0%)

Test set: Average loss: 0.5750, Accuracy: 62730/80711 (78%)



with more training data, the model almost always predicts wake becuase it's so overepresented, when i was running just 10000 periods, it was in the mid to low 90s for non-rem and rem accuracy but with all the data, it's 99% accurate for wake and in the 30s for non rem